# Detector de Faces, Gatos e Cachorros

Por: Elaine Cristina Meirelles e Gustavo Coelho

OBS. Todos os membros contribuíram no trabalho. Onde:
- Gustavo: refatoração e aplicação do padrão Private Class;
- Elaine: refatoração e criação dos testes unitários.

**Importando as bibliotecas necessárias**

In [1]:
import numpy as np
import os
import sys
import cv2
import urllib2
import urllib

**Detectando diretórios de Início e de Caffe**

In [2]:
home_directory = os.getenv("HOME")
caffe_root = os.path.join(home_directory, 'caffe')
sys.path.insert(0, os.path.join(caffe_root, 'python'))

import caffe

**Uma classe de ligação entre o reconhecimento de faces e animais foi criada**

In [3]:
class NetFace:
    
    def __init__(self):
        haarcascade_xml = "haarcascade_frontalface_alt.xml"
        self.classifier = cv2.CascadeClassifier(haarcascade_xml)
        
    def classifierImage(self,miniframe):
        return self.classifier.detectMultiScale(miniframe)

**Além disso, foi desenvolvida uma classe para encapsular o reconhecimento de face**

In [4]:
class Face:
    
    def __init__(self,net=NetFace()):
        self.net = net

    def detect(self,frame):
        height, width, depth = frame.shape

        # create grayscale version
        grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # equalize histogram
        cv2.equalizeHist(grayscale, grayscale)

        # detect objects
        DOWNSCALE = 4
        minisize = (frame.shape[1]/DOWNSCALE,frame.shape[0]/DOWNSCALE)
        miniframe = cv2.resize(frame, minisize)
        faces = self.net.classifierImage(miniframe)
        
        # show rectangles on the faces
        #if len(faces)>0:
        #    for i in faces:
        #        x, y, w, h = [ v*DOWNSCALE for v in i ]
        #        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
        #        cv2.imshow('frame',frame)
        #        
        #        cv2.waitKey(0)
        #        
        #        print("Faces: ", len(faces))
        
        return faces

**Utilização do Private Class Design Pattern, através da criação de uma classe de dados**

In [5]:
class NetData:

    def __init__(self,net,transformer,labels):
        self.transformer = transformer
        self.labels = labels
        self.net = net

    def get_transformer(self):
        return self.transformer

    def get_labels(self):
        return self.labels

    def get_net(self):
        return self.net

**Uma outra classe foi desenvolvida para encapsular o reconhecimento de classes de animais**

In [6]:
class Net:

    def __init__(self,net,transformer,labels):
        self.netData = NetData(net,transformer,labels)
        self.load_synset()
        
    def load_synset(self):
        f = open("synset_cats","r")
        self.cats = f.read().splitlines()
        f.close()
        
        f = open("synset_dogs","r")
        self.dogs = f.read().splitlines()
        f.close()

    def predict_imageNet(self,image_filename):
        image = caffe.io.load_image(image_filename)
        self.netData.get_net().blobs['data'].data[...] = self.netData.get_transformer().preprocess('data', image)

        # perform classification
        self.netData.get_net().forward()

        # obtain the output probabilities
        output_prob = self.netData.get_net().blobs['prob'].data[0]

        # sort top five predictions from softmax output
        top_inds = output_prob.argsort()[::-1][:5]

        predictions = zip(output_prob[top_inds], self.netData.get_labels()[top_inds])

        return predictions

    def result(self,img):
        predictions = self.predict_imageNet(img)
        total_dogs = 0
        total_cats = 0
        
        for per, cls in predictions:
            if cls.split()[0] in self.cats:
                total_cats += per

            elif cls.split()[0] in self.dogs:
                total_dogs += per
        
        return total_dogs*100,total_cats*100

**Classe necessária para gerenciamento dos tipos de entrada do usuário**

In [7]:
class Input:

    FACES = 1
    PREDICTION = 2

    def __init__(self, load=None):
        self.img = None
        if(load): self.resolve(load)

    def resolve(self,load):
        if(self.isFile(load)):
            self.fileResolver(load)	
        
        elif(self.isUrl(load)):
            self.urlResolver(load)

    def isFile(self,load):
        return os.path.isfile(load)

    def isUrl(self,load):
        try:
            urllib2.urlopen(load)
            return True

        except urllib2.HTTPError, e:
            return False
        
        except urllib2.URLError, e:
            return False

        return False

    def fileResolver(self,load):
        self.img = cv2.imread(load)
        self.load = load

    def urlResolver(self,load):
        image = urllib.URLopener()
        path = "Images"
        image.retrieve(load,path)
        image.close()
        self.fileResolver(path)
        
        return None

    def getImage(self,destination):
        if(destination == self.FACES):
            return self.img

        else:
            return self.load

**Classe responsável pela saída do resultado para o usuário**

In [8]:
class Output:

    def outFaces(self,faces):
        if(len(faces) > 0):
            print("Foram detectadas {0} faces.").format(str(len(faces)))
            print("As coordenadas das faces são: ")

            for face in faces:
                print(face)
        
        else:
            print("Não foram encontradas faces.")

        
    def outAnimals(self,dogs,cats):
        if(dogs > 0):
            print("A probabilidade de haver cães na imagem é de: {0}%.").format(str(dogs))

        else:
            print("Não há cães na imagem.")

        if(cats > 0):
            print("A probabilidade de haver gatos na imagem é de: {0}%.").format(str(cats))

        else:
            print("Não há gatos na imagem.")

** Método principal para rodar a aplicação**

In [9]:
def main(argv=sys.argv):
    mu = np.load(os.path.join(caffe_root, 'python','caffe','imagenet','ilsvrc_2012_mean.npy'))
    mu = mu.mean(1).mean(1)  

    model_weights = os.path.join(caffe_root, 'models','bvlc_reference_caffenet','bvlc_reference_caffenet.caffemodel')
    model_def = os.path.join(caffe_root, 'models', 'bvlc_reference_caffenet','deploy.prototxt')
    net = caffe.Net(model_def,model_weights,caffe.TEST)

    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', mu)      
    transformer.set_raw_scale('data', 255)      
    transformer.set_channel_swap('data', (2,1,0)) 

    labels_file = os.path.join(caffe_root, 'data','ilsvrc12','synset_words.txt')
    labels = np.loadtxt(labels_file, str, delimiter='\t') 

    f = Face()
    n = Net(net,transformer,labels)
    o = Output()
    
    if(len(argv) >= 2):
        argv.pop(0)
    
        for arg in argv:
            print(arg)
            
            d = Input(arg)
            o.outFaces(f.detect(d.getImage(Input.FACES)))
            
            dogs,cats = n.result(d.getImage(Input.PREDICTION))
            o.outAnimals(dogs,cats)

**Listando todas as images existentes na pasta Images**

In [10]:
image_directory = 'Images'
images = os.listdir(image_directory)

**Rodando a aplicação para cada uma das imagens encontradas**

In [11]:
for image_file in images:
    main([" ", os.path.join(image_directory, image_file)])
    print "\n"

Images/Cachorro-1.jpg
Não foram encontradas faces.
A probabilidade de haver cães na imagem é de: 44.1126599908%.
Não há gatos na imagem.


Images/Gato-1.jpg
Não foram encontradas faces.
Não há cães na imagem.
A probabilidade de haver gatos na imagem é de: 67.4296088517%.


Images/Familia-com-Cachorro.jpg
Foram detectadas 3 faces.
As coordenadas das faces são: 
[129  22  31  31]
[85 94 45 45]
[138  97  64  64]
A probabilidade de haver cães na imagem é de: 39.8055158556%.
Não há gatos na imagem.


Images/Gato-2.jpg
Não foram encontradas faces.
Não há cães na imagem.
A probabilidade de haver gatos na imagem é de: 99.996507098%.


Images/Pessoa.jpg
Foram detectadas 1 faces.
As coordenadas das faces são: 
[ 34  37 118 118]
Não há cães na imagem.
Não há gatos na imagem.


Images/Familia.jpg
Foram detectadas 2 faces.
As coordenadas das faces são: 
[76 12 28 28]
[104   8  36  36]
Não há cães na imagem.
Não há gatos na imagem.


Images/Cachorro-2.jpg
Foram detectadas 1 faces.
As coordenadas das

**Importando a biblioteca de teste unitário**

In [12]:
import unittest
from timeit import default_timer as timer

**Classe de teste para a detecção de faces**

In [16]:
class TestFace(unittest.TestCase):

    def test_1_face_detected (self):
        start = timer()
        image = os.path.join('Images', 'Pessoa.jpg')
        d = Input(image)
        
        faces = face.detect(d.getImage(Input.FACES))
        
        output.outFaces(faces)
        end =  timer()
        print ("Tempo de execução: %.2f\n" % (end-start))
        
        self.assertEqual(len(faces),1)
        
    def test_more_than_1_face_detected (self):
        start = timer()
        image = os.path.join('Images', 'Familia.jpg')
        d = Input(image)

        faces = face.detect(d.getImage(Input.FACES))
        
        output.outFaces(faces)
        
        end =  timer()
        print ("Tempo de execução: %.2f\n" % (end-start))
        
        self.assertEqual(len(faces),2)
        
    def test_no_face_detected (self):
        start = timer()
        image = os.path.join('Images', 'Gato-1.jpg')
        d = Input(image)
        
        faces = face.detect(d.getImage(Input.FACES))
        
        output.outFaces(faces)
        
        end =  timer()
        print ("Tempo de execução: %.2f\n" % (end-start))
        
        self.assertEqual(len(faces),0)

if __name__ == "__main__":
    # instaciating face and output classes
    face = Face()
    output = Output()
        
    suite = unittest.TestSuite()
    
    for method in dir(TestFace):
        if method.startswith("test"):
            suite.addTest(TestFace(method))
    unittest.TextTestRunner().run(suite)

...

Foram detectadas 1 faces.
As coordenadas das faces são: 
[ 34  37 118 118]
Tempo de execução: 0.05

Foram detectadas 2 faces.
As coordenadas das faces são: 
[76 12 28 28]
[104   8  36  36]
Tempo de execução: 0.01

Não foram encontradas faces.
Tempo de execução: 0.01




----------------------------------------------------------------------
Ran 3 tests in 0.089s

OK


**Classe de teste para a detecção de gatos e cachorros**

In [17]:
class TestAnimal(unittest.TestCase):
    
    def test_dog_detected(self):
        start = timer()
        image = os.path.join('Images', 'Cachorro-1.jpg')
        d = Input(image)
        
        dogs,cats = net_class.result(d.getImage(Input.PREDICTION))
        
        output.outAnimals(dogs,cats)
        end =  timer()
        print ("Tempo de execução: %.2f\n" % (end-start))
        
        self.assertGreaterEqual(dogs, 0)
        self.assertEqual(cats, 0)
        
    def test_cat_detected(self):
        start = timer()
        image = os.path.join('Images', 'Gato-1.jpg')
        d = Input(image)
        
        dogs,cats = net_class.result(d.getImage(Input.PREDICTION))
        
        output.outAnimals(dogs,cats)
        end =  timer()
        print ("Tempo de execução: %.2f\n" % (end-start))
        
        self.assertGreater(cats, 0)
        self.assertEqual(dogs, 0)
    
    def test_no_cat_no_dog_detected(self):
        start = timer()
        image = os.path.join('Images', 'Familia.jpg')
        d = Input(image)
        
        dogs,cats = net_class.result(d.getImage(Input.PREDICTION))
        
        output.outAnimals(dogs,cats)
        end =  timer()
        print ("Tempo de execução: %.2f\n" % (end-start))
        
        self.assertEqual(dogs, 0)
        self.assertEqual(cats, 0)

if __name__ == "__main__":
    mu = np.load(os.path.join(caffe_root, 'python','caffe','imagenet','ilsvrc_2012_mean.npy'))
    mu = mu.mean(1).mean(1)  

    model_weights = os.path.join(caffe_root, 'models','bvlc_reference_caffenet','bvlc_reference_caffenet.caffemodel')
    model_def = os.path.join(caffe_root, 'models', 'bvlc_reference_caffenet','deploy.prototxt')
    net = caffe.Net(model_def,model_weights,caffe.TEST)

    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', mu)      
    transformer.set_raw_scale('data', 255)      
    transformer.set_channel_swap('data', (2,1,0)) 

    labels_file = os.path.join(caffe_root, 'data','ilsvrc12','synset_words.txt')
    labels = np.loadtxt(labels_file, str, delimiter='\t') 

    # instanciating Net and Output
    net_class = Net(net,transformer,labels)
    output = Output()

    suite = unittest.TestSuite()

    for method in dir(TestAnimal):
        if method.startswith("test"):
            suite.addTest(TestAnimal(method))
    unittest.TextTestRunner().run(suite)

.

Não há cães na imagem.
A probabilidade de haver gatos na imagem é de: 67.4296088517%.
Tempo de execução: 1.37



.

A probabilidade de haver cães na imagem é de: 44.1126599908%.
Não há gatos na imagem.
Tempo de execução: 1.35



.

Não há cães na imagem.
Não há gatos na imagem.
Tempo de execução: 1.45




----------------------------------------------------------------------
Ran 3 tests in 4.178s

OK
